In [1]:
# This code emphasizes simplicity and brevity over robustness.
# Core commentary is in the write-up rather than this notebook.
# This is mostly didactic, not a production-ready LID library.

In [2]:
import math

def lid(radius, radius_bound):
    return 1 / math.log(radius_bound / radius)

In [3]:
# In one dimension,
# the expected radius for a radius_bound of one is 0.5,
# but that doesn't imply the converse:
# a measured radius of 0.5 withing a radius_bound of one
# doesn't imply the most likely dimensionality is one.
# There may be an interesting explanation of this.
assert lid(1/math.e, 1) == 1

In [4]:
def euclidean(xs, ys):  # Euclidean distance
    return math.sqrt(sum((x - y)**2 for x, y in zip(xs, ys)))

In [5]:
assert euclidean([3, 0], [0, 4]) == 5
# The behavior of zip means unexpected behavior can occur
# if len(xs) != len(ys). That's the kind of thing this code doesn't check.

In [6]:
def harmonic_mean(numbers):
    return len(numbers) / sum(1 / number for number in numbers)

In [7]:
assert harmonic_mean([3]) == 3
assert harmonic_mean([1, 4, 4]) == 2

In [8]:
def positive(numbers):
    return [number for number in numbers if number > 0]

In [9]:
assert positive([0, 1, 2, 3]) == [1, 2, 3]

In [10]:
def neighbors_lid(origin, points, k=1, distance=euclidean):
    radii = sorted(positive([distance(origin, point) for point in points]))
    return harmonic_mean([lid(radius, radii[k]) for radius in radii[:k]])

In [11]:
# neighbors_lid doesn't ensure that radii[k]
# is strictly greater than all radii[:k],
# which is needed for reasonable behavior.

# When using floats (as in the iris dataset)
# equal distances can give small differences
# as in: 0.8 - 0.7 > 0.7 - 0.6  # True
# which gives incorrectly huge LID.

In [12]:
import random

def cloud(dimensions, count=100, bounds=(-1, 1)):
    return [[random.uniform(*bounds) for _ in range(dimensions)]
            for _ in range(count)]

In [13]:
random.seed(80)

neighbors_lid([0]*7, cloud(7))

6.851551579378687

In [14]:
random.seed(12)

harmonic_mean([neighbors_lid([0]*7, cloud(7)) for _ in range(1000)])

7.006859728132216

In [15]:
def data_lid(points, k=1, distance=euclidean):
    return harmonic_mean([neighbors_lid(point, points, k=k, distance=distance)
                          for point in points])

In [16]:
# data_lid averages dimensionality over a whole dataset,
# but it could be that parts of a dataset have different dimensionalities.
# It might make sense to break up a dataset,
# or do a local weighted average to see
# how dimensionality varies across a UMAP visualization, for instance.

In [17]:
import sklearn.datasets

roll, _ = sklearn.datasets.make_swiss_roll(random_state=9)

In [18]:
data_lid(roll)

2.0706005383894444